# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# define API key
g_key = "G_KEY"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# laod csv file
weather_df = pd.read_csv('../output_data/weather_df.csv')

# checkpoint
weather_df.head()

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,Rikitea,-23.1203,-134.9692,77.23,73,96,12.59,PF,1612728440
1,Tuktoyaktuk,69.4541,-133.0374,-25.60,75,75,13.80,CA,1612728622
2,Georgiyevka,43.0369,74.7114,39.20,81,100,4.47,KZ,1612728622
3,Saint-Joseph,-21.3667,55.6167,80.60,74,90,4.61,RE,1612728137
4,Hobyo,5.3505,48.5268,78.10,78,0,17.29,SO,1612728370


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gkey
gmaps.configure(api_key=g_key)

# set variables
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity (%)"].astype(float)

# create heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

# show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.
  
* Drop any rows will null values.

In [4]:
# create new dataframe for ideal vacation cities
vacation_df = pd.DataFrame(weather_df)

# set conditions variables
max_temp = (vacation_df["Max Temp (F)"] <= 80) & (vacation_df["Max Temp (F)"] > 60)
humidity = vacation_df["Humidity (%)"] < 45
cloudiness = vacation_df["Cloudiness (%)"] <= 50
wind_speed = vacation_df["Wind Speed (MPH)"] <= 10

# drop rows that do not match conditions
vacation_df = vacation_df.loc[max_temp]
vacation_df = vacation_df[humidity]
vacation_df = vacation_df[cloudiness]

# reset index
vacation_df.reset_index()

<ipython-input-4-bf48b53f6f95>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacation_df = vacation_df[humidity]
<ipython-input-4-bf48b53f6f95>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vacation_df = vacation_df[cloudiness]


,index,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,39,Punta Arenas,-53.1500,-70.9167,71.60,23,0,13.80,CL,1612728463
1,132,Saint George,37.1041,-113.5841,60.01,25,1,4.61,US,1612728658
2,141,Umm Kaddadah,13.6017,26.6876,65.68,25,0,13.60,SD,1612728661
3,142,Taoudenni,22.6783,-3.9836,64.33,27,0,9.28,ML,1612728661
4,143,Azare,11.6765,10.1948,74.46,19,15,8.68,NG,1612728661
5,159,Pahrump,36.2083,-115.9839,61.00,22,0,3.00,US,1612728666
6,162,Aswān,24.0934,32.9070,64.47,44,0,13.09,EG,1612728659
7,219,Puerto Palomas,31.7667,-107.6167,66.20,9,1,18.41,MX,1612728683
8,263,Winslow,35.0242,-110.6974,61.00,11,1,10.36,US,1612728695
9,270,Okahandja,-21.9833,16.9167,79.00,33,4,8.05,NaN,1612728697


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# store into new df
hotel_df = pd.DataFrame(vacation_df)

# create new column for Hotel Name
hotel_df['Hotel Name']=''

# checkpoint print
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
39,Punta Arenas,-53.1500,-70.9167,71.60,23,0,13.80,CL,1612728463,
132,Saint George,37.1041,-113.5841,60.01,25,1,4.61,US,1612728658,
141,Umm Kaddadah,13.6017,26.6876,65.68,25,0,13.60,SD,1612728661,
142,Taoudenni,22.6783,-3.9836,64.33,27,0,9.28,ML,1612728661,
143,Azare,11.6765,10.1948,74.46,19,15,8.68,NG,1612728661,
159,Pahrump,36.2083,-115.9839,61.00,22,0,3.00,US,1612728666,
162,Aswān,24.0934,32.9070,64.47,44,0,13.09,EG,1612728659,
219,Puerto Palomas,31.7667,-107.6167,66.20,9,1,18.41,MX,1612728683,
263,Winslow,35.0242,-110.6974,61.00,11,1,10.36,US,1612728695,
270,Okahandja,-21.9833,16.9167,79.00,33,4,8.05,NaN,1612728697,


In [6]:
# set up parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# for loop to get hotel for each city
for index, row in hotel_df.iterrows():
    
    # get locations
    lat = row["Lat"]
    lng = row["Lng"]

    # set location for each loop
    params["location"] = f"{lat},{lng}"
    
    #set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # try loop to append hotel names to hotel_df
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    # fill Hotel Name with NaN if no hotels are found 
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = float('NaN')
    
# checkpoint print
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
39,Punta Arenas,-53.1500,-70.9167,71.60,23,0,13.80,CL,1612728463,Hotel Dreams Del Estrecho
132,Saint George,37.1041,-113.5841,60.01,25,1,4.61,US,1612728658,Best Western Plus Abbey Inn
141,Umm Kaddadah,13.6017,26.6876,65.68,25,0,13.60,SD,1612728661,NaN
142,Taoudenni,22.6783,-3.9836,64.33,27,0,9.28,ML,1612728661,NaN
143,Azare,11.6765,10.1948,74.46,19,15,8.68,NG,1612728661,Muk hostel
159,Pahrump,36.2083,-115.9839,61.00,22,0,3.00,US,1612728666,Best Western Pahrump Oasis
162,Aswān,24.0934,32.9070,64.47,44,0,13.09,EG,1612728659,Nuba Nile
219,Puerto Palomas,31.7667,-107.6167,66.20,9,1,18.41,MX,1612728683,Hotel San Carlos
263,Winslow,35.0242,-110.6974,61.00,11,1,10.36,US,1612728695,Best Western Plus Winslow Inn
270,Okahandja,-21.9833,16.9167,79.00,33,4,8.05,NaN,1612728697,Sylvanette Guesthouse


In [7]:
# drop cities with no hotels
hotel_df = hotel_df.dropna(how='any')

#checkpoint print
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
39,Punta Arenas,-53.1500,-70.9167,71.60,23,0,13.80,CL,1612728463,Hotel Dreams Del Estrecho
132,Saint George,37.1041,-113.5841,60.01,25,1,4.61,US,1612728658,Best Western Plus Abbey Inn
143,Azare,11.6765,10.1948,74.46,19,15,8.68,NG,1612728661,Muk hostel
159,Pahrump,36.2083,-115.9839,61.00,22,0,3.00,US,1612728666,Best Western Pahrump Oasis
162,Aswān,24.0934,32.9070,64.47,44,0,13.09,EG,1612728659,Nuba Nile
219,Puerto Palomas,31.7667,-107.6167,66.20,9,1,18.41,MX,1612728683,Hotel San Carlos
263,Winslow,35.0242,-110.6974,61.00,11,1,10.36,US,1612728695,Best Western Plus Winslow Inn
315,Rajanpur,29.1035,70.3250,63.23,26,0,7.94,PK,1612728710,Style Inn Shoes Rajanpur
344,Morro Bay,35.3658,-120.8499,75.20,20,1,5.75,US,1612728717,Inn at Morro Bay
377,San Patricio,28.0170,-97.5169,75.20,35,1,14.97,US,1612728728,"Motel 6 Sinton, TX"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# set up gmaps figure
fig = gmaps.figure()

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(gmaps.marker_layer(locations))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))